Dataset from [Kaggle](https://www.kaggle.com/c/aptos2019-blindness-detection/overview)

In [ ]:
import sys
sys.path.append("../src")  
from imports import *  
import importdata

#Import Data
importdata.main()
df_train = pd.read_csv('../data/raw/aptos2019-blindness-detection/train.csv')

In [ ]:
train_image_dir = '../data/raw/aptos2019-blindness-detection/train_images/'

class_labels = {
    0: 'No DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferative'
}

plt.figure(figsize=(20, 10))

for i in range(5):
    # Find the first image for the current class
    sample_id = df_train[df_train['diagnosis'] == i].iloc[0]['id_code']
    image_path = os.path.join(train_image_dir, f"{sample_id}.png")
    
    # Load the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert for matplotlib
    img = cv2.resize(img, (224, 224)) # Resize for consistent display
    
    # Display the image
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(f"Class: {i} ({class_labels[i]})")
    plt.axis('off')
    
plt.show()

In [ ]:
class_counts = df_train['diagnosis'].value_counts().sort_index()

# Plot the distribution
plt.figure(figsize=(10, 6))

# The corrected line is below:
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis", hue=class_counts.index, legend=False)

plt.title('Distribution of Diabetic Retinopathy Classes')
plt.xlabel('Diagnosis (Severity)')
plt.ylabel('Number of Images')
plt.xticks(ticks=[0, 1, 2, 3, 4], labels=['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative'])
plt.show()